In [0]:
import dlt
import json
from pyspark.sql.functions import from_json, expr, lit, col
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

In [0]:
catalog_name = "robin_huebner"
schema_name= "tfl_analytics"
volume_name = "bronze"
directory_name = "bike_point"

source_volume = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/{directory_name}/"

In [0]:
@dlt.table(
  comment="Loads JSON files from bronze into silver table"
)
def silver_bike_point():
  df = (spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("inferSchema", "true")
        .option("header", "true")
        .load(source_volume)
  )

  additional_properties_schema = ArrayType(
      StructType([
          StructField("key", StringType(), True),
          StructField("value", StringType(), True)
      ])
  )

  df = df.withColumn(
    "additionalProperties",
    from_json(col("additionalProperties"), additional_properties_schema)
  )

  # Define fields to extract from JSON
  fields_to_extract = [
      "TerminalName",
      "Installed",
      "Locked",
      "InstallDate",
      "RemovalDate",
      "Temporary",
      "NbBikes",
      "NbEmptyDocks",
      "NbDocks",
      "NbStandardBikes",
      "NbEBikes"
  ]

  # Define expressions for extraction from JSON
  expressions_extract = [
    f"filter(additionalProperties, x -> x.key = '{field}')[0].value as {field}"
    for field in fields_to_extract
  ]

  # Extract specified fields from JSON
  df = df.selectExpr("*", *expressions_extract)

  # Map each field to a Spark expression
  for field in fields_to_extract:
    df = df.withColumn(
        field,
        expr(f"filter(additionalProperties, x -> x.key = '{field}')[0].value as {field}")
    )


  # Define columns to keep and column names
  fields_to_rename = {
    "id": "bikepoint_id",
    "commonName": "bikepoint_name",
    "lat": "bikepoint_latitude",
    "lon": "bikepoint_longitude",
    "TerminalName": "terminal_name",
    "Installed": "installed",
    "Locked": "locked",
    "InstallDate": "install_date",
    "RemovalDate": "removal_date",
    "Temporary": "temporary",
    "NbBikes": "bike_count",
    "NbEmptyDocks": "empty_dock_count",
    "NbDocks": "dock_count",
    "NbStandardBikes": "standard_bike_count",
    "NbEBikes": "ebike_count",
  }

  # Define expression to filter and rename columns
  expressions_rename = [
    f"{old_name} as {new_name}"
    for old_name, new_name in fields_to_rename.items()
  ]

  # Filter and rename dataframe
  df = df.selectExpr(
    [*expressions_rename]
  )

  # Add audit information to the table
  df = (
    df.withColumn("source_system", lit("TFL"))
    .withColumn("ingestion_user", expr("CURRENT_USER()"))
    .withColumn("ingestion_timestamp", expr("CURRENT_TIMESTAMP()"))
  )
  return df

In [0]:
# # Load dataframe from volume
# df = spark.read.json("/Volumes/robin_huebner/tfl_analytics/bronze/bike_point/tfl_data.json")

# # Define fields to extract from JSON
# fields_to_extract = [
#     "TerminalName",
#     "Installed",
#     "Locked",
#     "InstallDate",
#     "RemovalDate",
#     "Temporary",
#     "NbBikes",
#     "NbEmptyDocks",
#     "NbDocks",
#     "NbStandardBikes",
#     "NbEBikes"
# ]

# # Define expressions for extraction from JSON
# expressions_extract = [
#   f"filter(additionalProperties, x -> x.key = '{field}')[0].value as {field}"
#   for field in fields_to_extract
# ]

# # Extract specified fields from JSON
# df = df.selectExpr("*", *expressions_extract)

# # Define columns to keep and column names
# fields_to_rename = {
#   "id": "bikepoint_id",
#   "commonName": "bikepoint_name",
#   "lat": "bikepoint_latitude",
#   "lon": "bikepoint_longitude",
#   "TerminalName": "terminal_name",
#   "Installed": "installed",
#   "Locked": "locked",
#   "InstallDate": "install_date",
#   "RemovalDate": "removal_date",
#   "Temporary": "temporary",
#   "NbBikes": "bike_count",
#   "NbEmptyDocks": "empty_dock_count",
#   "NbDocks": "dock_count",
#   "NbStandardBikes": "standard_bike_count",
#   "NbEBikes": "ebike_count",
# }

# # Define expression to filter and rename columns
# expressions_rename = [
#   f"{old_name} as {new_name}"
#   for old_name, new_name in fields_to_rename.items()
# ]

# # Filter and rename dataframe
# df = df.selectExpr(
#   [*expressions_rename]
# )

# # Add audit information to the table
# df = (
#   df.withColumn("source_system", lit("TFL"))
#   .withColumn("ingestion_user", expr("CURRENT_USER()"))
#   .withColumn("ingestion_timestamp", expr("CURRENT_TIMESTAMP()"))
# )

# # Write dataframe to silver
# df.write.mode("append").saveAsTable("silver_tfl_bike_point")




# display(df)